In [1]:
import get
import matplotlib.pyplot as plt
import dask
import dask.dataframe as dd
from dask.distributed import Client, progress
client = Client(n_workers=2, threads_per_worker=2, memory_limit='2GB')
client

Client Scheduler: tcp://127.0.0.1:50454 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 2 Cores: 4 Memory: 4.00 GB


In [2]:
SPY = get.daily('SPY', outputsize = 'full')

In [3]:
SPY = SPY.sort_values('timestamp', ignore_index=True)

In [4]:
SPY

,timestamp,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient
0,2000-03-09,137.2500,140.8750,136.1250,140.8750,96.8248,5500900,0.0,1.0
1,2000-03-10,140.1875,142.0000,139.5312,140.1250,96.3093,7924600,0.0,1.0
2,2000-03-13,136.6875,140.4687,135.6875,138.5937,95.2569,10540500,0.0,1.0
3,2000-03-14,139.2812,140.0937,136.1562,136.6250,93.9038,8263900,0.0,1.0
4,2000-03-15,136.8750,140.4375,136.0625,139.8125,96.0946,10300800,0.0,1.0
...,...,...,...,...,...,...,...,...,...
5028,2020-03-05,304.9800,308.4700,300.0100,302.4600,302.4600,185533970,0.0,1.0
5029,2020-03-06,293.1500,298.7800,290.2300,297.4600,297.4600,228315219,0.0,1.0
5030,2020-03-09,275.3000,284.1900,273.4500,274.2300,274.2300,305980554,0.0,1.0
5031,2020-03-10,284.6400,288.5200,273.5000,288.4200,288.4200,261412280,0.0,1.0


In [5]:
SPY = dd.from_pandas(SPY, npartitions = len(SPY.columns))

In [ ]:
%matplotlib qt
plt.figure()
plt.scatter(SPY.timestamp.compute(), SPY.adjusted_close.compute())
#plt.plot(TBBK['volume'])
#plt.plot(SPY["high"])
#plt.plot(SPY["low"])
#plt.plot(SPY["close"])
plt.title('SPY history')
plt.ylabel('Price (USD)')
plt.xlabel('Days')
#plt.legend(['Open','High','Low','Close'], loc='upper left')
plt.show()

Computer keeps crashing, but using a little research I found that the first crash is 2008.
Lets look at this a bit closer

In [6]:
type(SPY.timestamp.head()[0])#figure out formatting

str

In [52]:
#thats convenient
start_date = SPY[SPY.timestamp == '2008-10-01'].compute().index.values.astype(int).item()

In [53]:
end_date= SPY[SPY.timestamp == '2008-12-01'].compute().index.values.astype(int).item()

In [84]:
#kinda hacking this, if you know an easier way, please make a pull request
O_EIGHT = SPY.index.apply(lambda x: x[x.index.isin(range(start_date, end_date))], axis = 1, meta = SPY.dtypes)